In [2]:
import requests
import pandas as pd

API_KEY = "34ed2dc9566ad15743e1ef7eac40a2ca"
SPORT = "basketball_nba"
REGION = "us"  # us | uk | eu | au
MARKETS = "h2h,spreads"
URL = f"https://api.the-odds-api.com/v4/sports/{SPORT}/odds"

params = {
    "apiKey": API_KEY,
    "regions": REGION,
    "markets": MARKETS,
    "oddsFormat": "american"
}

response = requests.get(URL, params=params)

if response.status_code == 200:
    data = response.json()
    print(f"Fetched {len(data)} games.")
    df = pd.json_normalize(data)
    df.to_csv("../data/raw_odds.csv", index=False)
    df.head()
else:
    print("Error:", response.status_code, response.text)


Fetched 44 games.


In [3]:
import json

rows = []

for game in data:
    game_id = game["id"]
    home_team = game["home_team"]
    away_team = game["away_team"]
    commence_time = game["commence_time"]
    
    for bookmaker in game["bookmakers"]:
        book_name = bookmaker["title"]
        last_update = bookmaker["last_update"]
        
        for market in bookmaker["markets"]:
            if market["key"] == "spreads":
                outcomes = market["outcomes"]
                spread_home = None
                spread_away = None
                price_home = None
                price_away = None
                
                for o in outcomes:
                    if o["name"] == home_team:
                        spread_home = o["point"]
                        price_home = o["price"]
                    else:
                        spread_away = o["point"]
                        price_away = o["price"]
                        
                rows.append({
                    "game_id": game_id,
                    "home_team": home_team,
                    "away_team": away_team,
                    "bookmaker": book_name,
                    "spread_home": spread_home,
                    "spread_away": spread_away,
                    "price_home": price_home,
                    "price_away": price_away,
                    "commence_time": commence_time,
                    "last_update": last_update
                })

df_clean = pd.DataFrame(rows)
df_clean.to_csv("../data/odds_flat.csv", index=False)
df_clean.head()


,game_id,home_team,away_team,bookmaker,spread_home,spread_away,price_home,price_away,commence_time,last_update
0,bbde7751a144b98ed150d7a5f7dc8f87,Oklahoma City Thunder,Houston Rockets,DraftKings,-8.0,8.0,-110,-110,2025-10-21T23:30:00Z,2025-10-06T02:15:41Z
1,bbde7751a144b98ed150d7a5f7dc8f87,Oklahoma City Thunder,Houston Rockets,FanDuel,-8.5,8.5,-106,-114,2025-10-21T23:30:00Z,2025-10-06T02:14:06Z
2,bbde7751a144b98ed150d7a5f7dc8f87,Oklahoma City Thunder,Houston Rockets,MyBookie.ag,-8.0,8.0,-110,-110,2025-10-21T23:30:00Z,2025-10-06T02:15:34Z
3,bbde7751a144b98ed150d7a5f7dc8f87,Oklahoma City Thunder,Houston Rockets,LowVig.ag,-8.0,8.0,-106,-106,2025-10-21T23:30:00Z,2025-10-06T02:15:32Z
4,bbde7751a144b98ed150d7a5f7dc8f87,Oklahoma City Thunder,Houston Rockets,Bovada,-8.0,8.0,-105,-115,2025-10-21T23:30:00Z,2025-10-06T02:15:01Z
